In [ ]:
# =================================================================
# 1. IMPORTS AND INITIAL SETUP
# =================================================================
import pandas as pd
import numpy as np
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# =================================================================
# 2. HYPERPARAMETERS AND DEVICE CONFIGURATION
# =================================================================

# Configure device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model and training hyperparameters
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 128  # Increased for better context capture
BATCH_SIZE = 16
EPOCHS = 4
LEARNING_RATE = 3e-5

# =================================================================
# 3. DATA LOADING AND PREPROCESSING
# =================================================================

# Load the dataset
df = pd.read_csv("reviews.csv")

# Preprocessing function to clean text
def preprocess_text(text):
    if isinstance(text, str):
        # Remove non-alphabetic characters, convert to lowercase
        text = re.sub(r"[^a-zA-Z\s]", "", text.lower())
        # Standardize whitespace
        text = re.sub(r"\s+", " ", text).strip()
        return text
    return ""  # Return empty string for NaN or other non-string inputs

# Apply preprocessing
df['text'] = df['text'].apply(preprocess_text)

# Data Cleaning
df.dropna(inplace=True)
df = df[df['text'] != ''] # Remove rows with empty text after preprocessing
df['rating'] = df['rating'].astype(int)

# Assert that ratings are within the expected 1-5 range
assert df['rating'].min() >= 1 and df['rating'].max() <= 5, "Ratings are out of the 1-5 range."



In [ ]:
# =================================================================
# 4. DATA SPLITTING (TRAIN, VALIDATION, TEST)
# =================================================================

# First, split into training (80%) and testing (20%)
X_train, X_test, y_train, y_test = train_test_split(
    df['text'],
    df['rating'],
    test_size=0.2,
    random_state=42,
    stratify=df['rating']
)

# Then, split the training set into training (90% of original 80%) and validation (10% of original 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.1, # 10% of the training data will be for validation
    random_state=42,
    stratify=y_train
)

# Shift labels from 1-5 to 0-4 for PyTorch/BERT
y_train = y_train - 1
y_val = y_val - 1
y_test = y_test - 1


# =================================================================
# 5. PYTORCH DATASET AND DATALOADERS
# =================================================================

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts.iloc[item]
        label = self.labels.iloc[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# Create Datasets
train_dataset = SentimentDataset(X_train, y_train, tokenizer, MAX_LEN)
val_dataset = SentimentDataset(X_val, y_val, tokenizer, MAX_LEN)
test_dataset = SentimentDataset(X_test, y_test, tokenizer, MAX_LEN)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# =================================================================
# 6. MODEL, OPTIMIZER, AND LOSS FUNCTION SETUP
# =================================================================

# Load BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=5)
model = model.to(device)

# Calculate class weights to handle data imbalance
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define loss function with class weights
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)



In [ ]:
# =================================================================
# 7. TRAINING AND EVALUATION FUNCTIONS
# =================================================================

def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler):
    model.train()
    total_loss = 0
    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Helps prevent exploding gradients
        optimizer.step()
        scheduler.step()
        
    return total_loss / len(data_loader)

def evaluate_model(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
    return accuracy, precision, recall, f1, all_labels, all_preds


In [ ]:
# =================================================================
# 8. MAIN TRAINING LOOP
# =================================================================

best_accuracy = 0
best_model_state = None

for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch + 1}/{EPOCHS} ---")
    
    avg_train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device, scheduler)
    print(f"Average Training Loss: {avg_train_loss:.4f}")
    
    val_accuracy, val_precision, val_recall, val_f1, _, _ = evaluate_model(model, val_loader, device)
    print(f"Validation Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")

    # Save the model if it has the best validation accuracy so far
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_model_state = model.state_dict()
        print(f"New best model saved with validation accuracy: {best_accuracy:.4f}")

# Load the best performing model state for final evaluation
model.load_state_dict(best_model_state)


In [ ]:
# =================================================================
# 9. FINAL EVALUATION ON TEST SET
# =================================================================
print("\n--- Final Evaluation on Test Set ---")
test_accuracy, test_precision, test_recall, test_f1, test_actual, test_preds = evaluate_model(model, test_loader, device)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-Score: {test_f1:.4f}")

# Plotting the confusion matrix
def plot_confusion_matrix(y_true, y_pred, model_name):
    # Remember to convert labels back to 1-5 for plotting
    y_true_plot = [i + 1 for i in y_true]
    y_pred_plot = [i + 1 for i in y_pred]
    cm = confusion_matrix(y_true_plot, y_pred_plot)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(1, 6), yticklabels=range(1, 6))
    plt.xlabel('Predicted Rating')
    plt.ylabel('Actual Rating')
    plt.title(f'{model_name} Confusion Matrix')
    plt.show()

print("\nBERT Confusion Matrix on Test Set:")
plot_confusion_matrix(test_actual, test_preds, model_name="BERT")


In [ ]:
# =================================================================
# 10. INFERENCE FUNCTION
# =================================================================
def predict_review(review_text):
    model.eval()
    
    # Preprocess the user's review text
    processed_text = preprocess_text(review_text)

    encoding = tokenizer.encode_plus(
        processed_text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        _, prediction = torch.max(outputs.logits, dim=1)
        
    # Convert prediction from 0-4 back to 1-5 rating
    return prediction.item() + 1

# Example of how to use the inference function
user_review = input("Please enter your review: ")
predicted_rating = predict_review(user_review)
print(f"The review is classified as: {predicted_rating}-star")